# Creates an MS-COCO style dataset for powerpoint

In [292]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</s}tyle>"))

In [365]:
# win32 com
#http://www.s-anand.net/blog/automating-powerpoint-with-python/
#https://docs.microsoft.com/en-us/office/vba/api/powerpoint.fillformat.visible
import win32com.client
import os
import random
import numpy as np
import PIL
from tqdm import tqdm
import random 
import lib
from natsort import natsorted

In [367]:
Application = win32com.client.Dispatch('PowerPoint.Application')
slide_height, slide_width = 540, 720
IMAGE_RES_DIVISOR = 1

In [3]:
part_x, part_y = (slide_height/IMAGE_RES_DIVISOR)/32, (slide_width/IMAGE_RES_DIVISOR)/32 
print(part_x*part_y, part_x, part_y)

379.6875 16.875 22.5


In [372]:
from importlib import reload
reload(lib)
reload(lib.ppt_interface)
from lib.ppt_interface import PPT_shapes, add_shape, add_line, add_title, add_textBox, convert_line_xyhw_to_points
shape_manager = PPT_shapes()

In [349]:
# Alright, lets generate some training data!

choices = [
             'TextBox',
             'StraightConnector',
             'Rectangle',
             'LeftArrow',
             'Chevron',
             'Oval',
             'Donut',
             'Isosceles Triangle',
        ]


def make_training_data_slide():
    slide = Presentation.Slides.Add(1, 11)
    
    # Put some text in the title
    slide.shapes.Title.TextFrame.TextRange.Text = "Title"
    
    
    num_shapes = np.random.randint(5,10) # MAX_SEQ_LEN includes start and end token
    
    segments = [{'x1':0.1, 'x2':0.2, 'y1':0.1, 'y2':0.4}, {'x1':0.3, 'x2':0.5, 'y1':0.1, 'y2':0.4}, {'x1':0.7, 'x2':0.9, 'y1':0.1, 'y2':0.4}, 
               {'x1':0.1, 'x2':0.2, 'y1':0.5, 'y2':0.9}, {'x1':0.3, 'x2':0.5, 'y1':0.5, 'y2':0.9} , {'x1':0.7, 'x2':0.9, 'y1':0.5, 'y2':0.9}]
    
    for s in range(0,num_shapes):
        choice = np.random.choice(choices)
        
        info = shape_manager.shapes[choice]
        if info[1] == 'line':
            # either horizontal or vertical only
            if np.random.uniform() < 0.5: # vertical
                x1, y1 = np.random.uniform(0.2,0.9), np.random.uniform(0.2,0.5)
                x2 = x1
                y2 = y1 + min(np.random.uniform(0.3,0.6), 1-y1)
            else: # vertical
                x1, y1 = np.random.uniform(0.2,0.5), np.random.uniform(0.2,0.9)
                y2 = y1
                x2 = x1 + min(np.random.uniform(0.3,0.6), 1-x1)
            
            x1,x2, y1,y2 = x1*slide_width, x2*slide_width, y1*slide_height, y2*slide_height
            add_line(slide, x1,y1,x2,y2)
        else:
            x = np.random.uniform(0.2,0.8)
            y = np.random.uniform(0.2,0.7)
            h = min(np.random.uniform(0.1,0.3), 1-y)
            w = min(np.random.uniform(0.1,0.3), 1-x) # don't want it bigger than the frame

            x,w, y,h = x*slide_width, w*slide_width, y*slide_height, h*slide_height

            rgb = np.zeros(3) # default 0
            border = 0 #default 0 
            fill = 1# np.random.randint(2)
            if fill == 1:
                col = np.random.randint(3)
                rgb[col] = 1 # literally only do r, g or b at the moment. This isn't a color accuracy test
            else:
                border = 1 # if not filled, then do a border
            text = 0# np.random.randint(2)

            ####################################
            if info[1] == 'textBox':
                add_textBox(slide, x,y,h,w)
            if info[1] == 'shape':
                add_shape(slide, info[0], x,y,h,w, rgb, border, fill,text)
        
    return slide

In [323]:
# Presentation = Application.Presentations.Add()
# 

StraightConnector
Chevron
StraightConnector
StraightConnector
Left Arrrow
TextBox


In [355]:
slide = make_training_data_slide()

In [4]:
#Presentation = Application.Presentations.Open(os.getcwd()+ '\\Presentation.pptx') # need full path with win22com
# Create a new ppt deck


# Make the deck of examples

In [341]:
def try_dir(path):
    try:
        os.makedirs(path)
    except Exception as e:
        print(e)
        pass


In [342]:
train_path =  os.getcwd()+ '\\data\\rgb_ppt\\train'
test_path =  os.getcwd()+ '\\data\\rgb_ppt\\val'
path = train_path
deck_path = path + '/decks/'
try_dir(deck_path)


[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Sholto Douglas\\Desktop\\artificial_graphics\\data\\rgb_ppt\\train/decks/'


In [362]:
existing_presos = len(os.listdir(path+'/decks/'))
N_PRESOS = existing_presos+1
for p in tqdm(range(existing_presos, N_PRESOS)):
    Presentation = Application.Presentations.Add()
    for i in tqdm(range(0,10)):
        make_training_data_slide() 
    Presentation.saveAs(path+f'/decks/{p}')
    Presentation.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.79s/it]


In [385]:
images = []
annotations = []

counter = 0
anno_counter = 0
for p in natsorted(os.listdir(path+'/decks/')):
    if '~' in p: # don't want the intermediate checkpoints
        continue
    Presentation = Application.Presentations.Open(f'{path}\\decks\\' + p)
    # Make a dir for the images from tha
    for i,slide in tqdm(enumerate(Presentation.Slides)):
        readout = read_slide(slide)
        slide.Export(path+f"/{counter}.jpg", 'JPG', slide_width, slide_height)
        images.append({
            "id": counter,
            "file_name": f"{counter}.jpg",
            "width": slide_width,
            "height": slide_height,
            "date_captured":  "2021-06-20 01:55:23.687250",
            "license": 1,
            "coco_url": "",
            "flickr_url": "",
        })
        for anno in readout:
            annotations.append({
                "id": anno_counter,
                "image_id": counter,
                "category_id": anno['class'],
                "iscrowd": 0,
                "area": anno['height']*anno['width'],
                "bbox": [anno['left'], anno['top'], anno['width'], anno['height']],
                "segmentation": [],
                
            })
            anno_counter += 1
        
        counter += 1

10it [00:02,  3.63it/s]
10it [00:03,  3.22it/s]


In [374]:
natsorted(os.listdir(path+'/decks/'))

['0.pptx', '1.pptx', '~$0.pptx', '~$1.pptx']

# Save images to disk

In [386]:
import json
data =  {
    "info":           {
                      "description": "PPTAI",
                      "url": "sholtodouglas.github.io",
                      "version": "0.1.0",
                      "year": 2021,
                      "contributor": "sholtodouglas",
                      "date_created": "2021-06-20 01:55:39.314230"
                      },
    "licenses": [
                  {
                      "id": 1,
                      "name": "Mine",
                      "url": "Mine"
                  }
                ],
    "categories" : [],
    "images": images,
    "annotations": annotations,
}

for name, ID in shape_manager.classIDs.items():
    data['categories'].append({'id': ID, 'name':name, 'supercategory':"N/A"})
json_object = json.dumps(data, indent = 4)  
print(json_object) 

{
    "info": {
        "description": "PPTAI",
        "url": "sholtodouglas.github.io",
        "version": "0.1.0",
        "year": 2021,
        "contributor": "sholtodouglas",
        "date_created": "2021-06-20 01:55:39.314230"
    },
    "licenses": [
        {
            "id": 1,
            "name": "Mine",
            "url": "Mine"
        }
    ],
    "categories": [
        {
            "id": 0,
            "name": "Title",
            "supercategory": "N/A"
        },
        {
            "id": 1,
            "name": "TextBox",
            "supercategory": "N/A"
        },
        {
            "id": 2,
            "name": "StraightConnector",
            "supercategory": "N/A"
        },
        {
            "id": 3,
            "name": "Rectangle",
            "supercategory": "N/A"
        },
        {
            "id": 4,
            "name": "LeftArrow",
            "supercategory": "N/A"
        },
        {
            "id": 5,
            "name": "Chevron",
      

In [ ]:
gsutil -m cp data/test/tf_records/*.tfrecords gs://artificial_graphics/data/test/tf_records

In [388]:
with open(f'{path}//custom_train.json', 'w') as f:
    json.dump(data, f)

In [387]:
f'{path}//custom_train.json'

'C:\\Users\\Sholto Douglas\\Desktop\\artificial_graphics\\data\\rgb_ppt\\train//custom_train.json'